In [0]:
query = '''
        select * from NxEnq_ResultadoEnquadramentoHist
        where status = 1
        data = '2025-01-29'
        order by data desc
        '''

token =  dbutils.secrets.get(scope="akvdesafiokinea", key="pwd-nexxus-sim")

remote_table = (spark.read \
  .format("sqlserver") \
  .option("host", "knanexxus.database.windows.net") \
  .option("user", "desafio_user") \
  .option("password", token) \
  .option("database", "sqldb-nexxus-sim-001") \
  .option("query", "query") # (if schemaName not provided, default to "dbo") \
  .load()
)

display(remote_table)

In [0]:
%sql
-- select * from desafio_kinea.boletagem_cp.nxenq_resultadoenquadramentohist
--     where status = 1
--     and DataHoraVersao = (select MAX(DataHoraVersao) from desafio_kinea.boletagem_cp.nxenq_resultadoenquadramentohist where status = 1);

select * from desafio_kinea.boletagem_cp.nxenq_regrasporconcentracao;

-- SELECT 
--     desafio_kinea.boletagem_cp.coluna,  -- Coluna da tabela origem
--     COALESCE(a.id, b.id) AS id,  -- ID correspondente das tabelas A ou B
--     COALESCE(a.outra_coluna, b.outra_coluna) AS outra_coluna -- Exemplo de outra coluna relevante
-- FROM origem o
-- LEFT JOIN A a 
--     ON a.id = CAST(SUBSTRING(o.coluna FROM '[0-9]+') AS INTEGER) 
--     AND o.coluna LIKE 'L%'  -- Garante que apenas 'L' case com A
-- LEFT JOIN B b 
--     ON b.id = CAST(SUBSTRING(o.coluna FROM '[0-9]+') AS INTEGER) 
--     AND (o.coluna LIKE 'CM%' OR o.coluna LIKE 'CD%'); -- Garante que 'CM' ou 'CD' casem com B

In [0]:
%sql

WITH UltimoHistorico AS (
    SELECT MAX(DataHoraVersao) AS UltimaData
    FROM desafio_kinea.boletagem_cp.nxenq_resultadoenquadramentohist
    WHERE status = 1
)

SELECT 
    nxe.IdFundo as IdFundo,
    nxe.IdRegra AS IdRegra_resultado_enquadramento,  
    nxe.TipoEnquadramento as TipoEnquadramento,
    nxe.SaldoBaseCalculo as SaldoBaseCalculo,
    nxe.ValorExposicao as ValorExposicao,
    nxe.SaldoObjeto as SaldoObjeto,
    nxe.LimiteMin as LimiteMin,
    nxe.LimiteMax as LimiteMax,
    nxr.IdRegra as IdRegra_nexusregras,
    nxr.ValorMin as ValorMin_nexusregras,
    nxr.ValorMax as ValorMax_nexusregras,
    nxr.Descricao as Descricao_nexusregras,
    nxr.DescricaoDetalhada as DescricaoDetalhada_nexusregras,
    nxrc.IdRegra as IdRegra_concentracao,
    nxrc.Descricao as Descricao_concentracao,
    nxrc.LimiteMin as LimiteMin_concentracao,
    nxrc.LimiteMax as LimiteMax_concentracao,
    nxrc.DescricaoDetalhada as DescricaoDetalhada_concentracao
FROM desafio_kinea.boletagem_cp.nxenq_resultadoenquadramentohist nxe
INNER JOIN UltimoHistorico uh 
    ON nxe.DataHoraVersao = uh.UltimaData  
LEFT JOIN desafio_kinea.boletagem_cp.nxenq_regras nxr 
    ON nxr.IdRegra = CAST(regexp_replace(nxe.IdRegra, '[^0-9]', '') AS INT) 
    AND nxe.IdRegra LIKE 'L%'  
LEFT JOIN desafio_kinea.boletagem_cp.nxenq_regrasporconcentracao nxrc
    ON nxrc.IdRegra = CAST(regexp_replace(nxe.IdRegra, '[^0-9]', '') AS INT)  
    AND (nxe.IdRegra LIKE 'CM%' OR nxe.IdRegra LIKE 'CD%')
where nxe.status = 1;

In [0]:
%sql
-- Query similar à anterior, mas com também fazendo join com a tabela de memoria de cálculo
--OBS Essa query não funciona por que as datas da tabela de memória estão desatualizadas (apenas de 2025 apenas)


WITH UltimoHistorico AS (
    SELECT MAX(DataHoraVersao) AS UltimaData
    FROM desafio_kinea.boletagem_cp.nxenq_resultadoenquadramentohist
    WHERE status = 1
)

SELECT 
    nxe.IdFundo as IdFundo,
    nxe.IdRegra AS IdRegra_resultado_enquadramento,  
    nxe.TipoEnquadramento as TipoEnquadramento,
    nxe.SaldoBaseCalculo as SaldoBaseCalculo,
    nxe.ValorExposicao as ValorExposicao,
    nxe.SaldoObjeto as SaldoObjeto,
    nxe.LimiteMin as LimiteMin,
    nxe.LimiteMax as LimiteMax,
    nxr.IdRegra as IdRegra_nexusregras,
    nxr.ValorMin as ValorMin_nexusregras,
    nxr.ValorMax as ValorMax_nexusregras,
    nxr.Descricao as Descricao_nexusregras,
    nxr.DescricaoDetalhada as DescricaoDetalhada_nexusregras,
    nxrc.IdRegra as IdRegra_concentracao,
    nxrc.Descricao as Descricao_concentracao,
    nxrc.LimiteMin as LimiteMin_concentracao,
    nxrc.LimiteMax as LimiteMax_concentracao,
    nxrc.DescricaoDetalhada as DescricaoDetalhada_concentracao,
    nxmc.MemoriaCalculo as MemoriaCalculo
FROM desafio_kinea.boletagem_cp.nxenq_resultadoenquadramentohist nxe
INNER JOIN UltimoHistorico uh 
    ON nxe.DataHoraVersao = uh.UltimaData  
LEFT JOIN desafio_kinea.boletagem_cp.nxenq_regras nxr 
    ON nxr.IdRegra = CAST(regexp_replace(nxe.IdRegra, '[^0-9]', '') AS INT) 
    AND nxe.IdRegra LIKE 'L%'  
LEFT JOIN desafio_kinea.boletagem_cp.nxenq_regrasporconcentracao nxrc
    ON nxrc.IdRegra = CAST(regexp_replace(nxe.IdRegra, '[^0-9]', '') AS INT)  
    AND (nxe.IdRegra LIKE 'CM%' OR nxe.IdRegra LIKE 'CD%')
LEFT JOIN desafio_kinea.boletagem_cp.nxenq_memoriacalculo nxmc --join com tabela de memoria de cálculo
    ON nxe.DataHoraVersao = nxmc.DataHoraVersao  --join pela hora e pelo ID da regra
    AND nxe.IdRegra = nxmc.IdRegra
where nxe.status = 1;

In [0]:
%sql
SELECT 
  *
FROM desafio_kinea.boletagem_cp.nxenq_resultadoenquadramentohist AS nxe
INNER JOIN desafio_kinea.boletagem_cp.nxenq_memoriacalculo AS nxmc 
  ON nxe.DataHoraVersao = nxmc.DataHoraVersao  --join pela hora e pelo ID da regra
  AND nxe.IdRegra = nxmc.IdRegra
WHERE nxe.Status = 1;

In [0]:
%sql
SELECT 
 nxmc.DataHoraVersao,
 nxmc.IdRegra,
 nxmc.IdFundo,
 nxmc.MemoriaCalculo,
 nxe.SaldoBaseCalculo,
 nxe.ValorExposicao,
 nxe.SaldoObjeto
FROM desafio_kinea.boletagem_cp.nxenq_memoriacalculo AS nxmc
JOIN (
  SELECT 
    IdRegra,
    DataHoraVersao,
    IdFundo,
    IdRegra,
    SaldoBaseCalculo,
    ValorExposicao,
    SaldoObjeto
  FROM desafio_kinea.boletagem_cp.nxenq_resultadoenquadramentohist
  WHERE Status = 1
) AS nxe
ON 
nxmc.DataHoraVersao = nxe.DataHoraVersao
AND nxmc.IdRegra = nxe.IdRegra

In [0]:
%sql
SELECT DataHoraVersao FROM desafio_kinea.boletagem_cp.nxenq_resultadoenquadramentohist
ORDER BY DataHoraVersao ASC
LIMIT 10

In [0]:
%sql
SELECT DataHoraVersao FROM desafio_kinea.boletagem_cp.nxenq_memoriacalculo
ORDER BY DataHoraVersao DESC
LIMIT 10

In [0]:
%sql
SELECT 
  *
FROM desafio_kinea.boletagem_cp.nxenq_memoriacalculo AS nxmc2 --tabela de memória de calculo do nexxus
INNER JOIN desafio_kinea.boletagem_cp.nxenq_resultadoenquadramentohist AS nxe2--inner join com tabela de histórico do nexxus
ON nxmc2.DataHoraVersao = nxe2.DataHoraVersao --join nas datas serem iguais
AND nxmc2.IdRegra = nxe2.IdRegra --e na regra ser igual
WHERE nxe2.Status = 1 --filtra pelas entradas com desenquadramento